# 第33章: 混合ガウスモデル（GMM）

## 📋 この章で学ぶこと

この章を終えると、以下ができるようになります：

- [ ] 多峰性分布の概念を理解できる
- [ ] 混合ガウスモデル（GMM）の式を説明できる
- [ ] GMMでデータを生成できる
- [ ] GMMのパラメータ推定の難しさを理解できる

## 🎯 前提知識

この章を学ぶには以下の知識が必要です：

- ✅ Notebook 30-32（正規分布と最尤推定の基礎）
- ✅ 多次元正規分布の確率密度関数

⏱️ **推定学習時間**: 120-150分  
📊 **難易度**: ★★★☆☆（中級）  
🎓 **カテゴリ**: 実践

---

## 🌟 はじめに

前章まで、単一の正規分布でデータをモデル化する方法を学びました。しかし、現実のデータは単一の正規分布では表現できないことが多いです。

### 🤔 単一の正規分布の限界

例えば、以下のようなデータを考えてみましょう：

- 男性と女性の身長データ（2つのピークがある）
- 複数の顧客セグメント（異なる購買パターン）
- 異なるクラスのデータが混在している状況

このようなデータは**多峰性（multimodal）**を持ち、単一の正規分布ではうまくモデル化できません。

**混合ガウスモデル（Gaussian Mixture Model, GMM）**は、複数の正規分布を組み合わせることで、このような複雑なデータ分布を表現できます。

### 🤔 なぜGMMが重要なのか？

1. **VAEへの橋渡し**: GMMは「潜在変数を持つ生成モデル」の最も基本的な例
2. **EMアルゴリズムの導入**: 次章で学ぶEMアルゴリズムの典型的な応用例
3. **クラスタリング**: 教師なし学習でのクラスタリング手法として使われる
4. **生成モデルの基礎**: より複雑な生成モデルの理解に必要

In [ ]:
# ============================================================
# 環境設定
# 必要なライブラリをインポートし、グラフの表示設定を行います
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.mixture import GaussianMixture
import warnings

# 警告を非表示
warnings.filterwarnings('ignore')

# グラフスタイルの設定
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# 日本語フォント設定
import matplotlib.font_manager as fm

def setup_japanese_font():
    """日本語フォントを設定する"""
    japanese_fonts = [
        'Hiragino Sans', 'Hiragino Maru Gothic Pro', 'AppleGothic',
        'Yu Gothic', 'MS Gothic',
        'Noto Sans CJK JP', 'IPAexGothic', 'TakaoPGothic',
    ]
    available_fonts = set(f.name for f in fm.fontManager.ttflist)
    for font in japanese_fonts:
        if font in available_fonts:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            return font
    return None

font_used = setup_japanese_font()
if font_used:
    print(f"日本語フォント: {font_used}")

# 再現性のための乱数シード
np.random.seed(42)

print("✅ ライブラリのインポート完了")

---

## 1. 多峰性分布

### 🤔 単峰性 vs 多峰性

- **単峰性（Unimodal）**: 1つのピークを持つ分布（例: 単一の正規分布）
- **多峰性（Multimodal）**: 複数のピークを持つ分布（例: 男女混合の身長データ）

多峰性分布を単一の正規分布でモデル化すると、実際のデータ構造を正しく捉えられません。

In [ ]:
# ============================================================
# 多峰性分布の例
# 単一の正規分布では表現できないデータを可視化します
# ============================================================

np.random.seed(42)

# 2つの異なるグループからデータを生成
# グループ1: 男性の身長（平均171cm）
# グループ2: 女性の身長（平均158cm）

n_male = 300
n_female = 300

heights_male = np.random.normal(171, 6, n_male)
heights_female = np.random.normal(158, 5, n_female)

# 混合データ
heights_mixed = np.concatenate([heights_male, heights_female])

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 左: 各グループの分布（別々に表示）
axes[0].hist(heights_male, bins=30, density=True, alpha=0.7, color='steelblue', 
             edgecolor='black', label='男性')
axes[0].hist(heights_female, bins=30, density=True, alpha=0.7, color='coral', 
             edgecolor='black', label='女性')
axes[0].set_xlabel('身長 (cm)', fontsize=12)
axes[0].set_ylabel('密度', fontsize=12)
axes[0].set_title('各グループの分布（ラベルあり）', fontsize=14, fontweight='bold')
axes[0].legend()

# 中央: 混合データ（ラベルなし）
axes[1].hist(heights_mixed, bins=40, density=True, alpha=0.7, color='purple', 
             edgecolor='black')
axes[1].set_xlabel('身長 (cm)', fontsize=12)
axes[1].set_ylabel('密度', fontsize=12)
axes[1].set_title('混合データ（ラベルなし）\n→ 多峰性分布', fontsize=14, fontweight='bold')

# 右: 単一正規分布でフィットした場合
mu_single = np.mean(heights_mixed)
sigma_single = np.std(heights_mixed)

axes[2].hist(heights_mixed, bins=40, density=True, alpha=0.7, color='gray', 
             edgecolor='black', label='実データ')

x = np.linspace(130, 200, 1000)
single_gaussian = stats.norm.pdf(x, mu_single, sigma_single)
axes[2].plot(x, single_gaussian, 'r-', linewidth=3, 
             label=f'単一正規分布\nN({mu_single:.1f}, {sigma_single:.1f}²)')

axes[2].set_xlabel('身長 (cm)', fontsize=12)
axes[2].set_ylabel('密度', fontsize=12)
axes[2].set_title('単一正規分布でのフィット\n→ うまくいかない！', fontsize=14, fontweight='bold')
axes[2].legend()

plt.tight_layout()
plt.show()

print("💡 ポイント: 多峰性データを単一の正規分布でモデル化すると、")
print("   2つのピークの間（存在しない領域）に確率が割り当てられてしまいます")

---

## 2. 混合ガウスモデル（GMM）の定義

### 📊 GMMとは？

**混合ガウスモデル（Gaussian Mixture Model, GMM）**は、複数の正規分布の重み付き和として定義されます：

$$p(\mathbf{x}) = \sum_{k=1}^{K} \pi_k \mathcal{N}(\mathbf{x} | \boldsymbol{\mu}_k, \Sigma_k)$$

ここで：
- $K$: 混合成分の数（クラスタ数）
- $\pi_k$: 混合係数（各成分の重み）、$\sum_{k=1}^{K} \pi_k = 1$、$\pi_k \geq 0$
- $\boldsymbol{\mu}_k$: 第$k$成分の平均ベクトル
- $\Sigma_k$: 第$k$成分の共分散行列

### 📊 GMMのパラメータ

| パラメータ | 説明 | 個数（d次元、K成分の場合）|
|-----------|------|---------------------------|
| $\pi_k$ | 混合係数 | K-1（合計1の制約） |
| $\boldsymbol{\mu}_k$ | 平均ベクトル | K × d |
| $\Sigma_k$ | 共分散行列 | K × d × (d+1)/2（対称性）|

In [ ]:
# ============================================================
# GMMの確率密度関数
# 複数の正規分布を重み付きで合成します
# ============================================================

def gmm_pdf(x, weights, means, stds):
    """
    1次元GMMの確率密度関数
    
    Parameters:
    -----------
    x : array-like
        評価点
    weights : array-like
        各成分の重み（合計1）
    means : array-like
        各成分の平均
    stds : array-like
        各成分の標準偏差
    
    Returns:
    --------
    array
        確率密度
    """
    pdf = np.zeros_like(x, dtype=float)
    for w, mu, sigma in zip(weights, means, stds):
        pdf += w * stats.norm.pdf(x, mu, sigma)
    return pdf

# 例: 身長データのGMM（2成分）
weights = [0.5, 0.5]  # 男女半々
means = [171, 158]     # 平均身長
stds = [6, 5]          # 標準偏差

x = np.linspace(130, 200, 1000)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: 各成分と混合分布
ax = axes[0]

# 各成分を描画
for i, (w, mu, sigma) in enumerate(zip(weights, means, stds)):
    component = w * stats.norm.pdf(x, mu, sigma)
    ax.fill_between(x, component, alpha=0.4, label=f'成分{i+1}: π={w}, μ={mu}, σ={sigma}')
    ax.plot(x, component, linewidth=2)

# 混合分布
gmm = gmm_pdf(x, weights, means, stds)
ax.plot(x, gmm, 'k-', linewidth=3, label='GMM（混合分布）')

ax.set_xlabel('身長 (cm)', fontsize=12)
ax.set_ylabel('確率密度', fontsize=12)
ax.set_title('GMMの構成: 2つの正規分布の重み付き和', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)

# 右: データとGMMのフィット
ax = axes[1]
ax.hist(heights_mixed, bins=40, density=True, alpha=0.7, color='gray', 
        edgecolor='black', label='実データ')
ax.plot(x, gmm, 'r-', linewidth=3, label='GMM')
ax.plot(x, single_gaussian, 'b--', linewidth=2, label='単一正規分布')

ax.set_xlabel('身長 (cm)', fontsize=12)
ax.set_ylabel('密度', fontsize=12)
ax.set_title('GMM vs 単一正規分布', fontsize=14, fontweight='bold')
ax.legend()

plt.tight_layout()
plt.show()

print("💡 GMMは多峰性データを正確にモデル化できます！")

---

## 3. 潜在変数とGMM

### 🤔 潜在変数とは？

GMMは**潜在変数（Latent Variable）**を持つモデルとして理解できます。

**潜在変数 $z$**: 各データ点がどの成分から生成されたかを示す変数

$$z \in \{1, 2, ..., K\}$$

### 📊 GMMの生成過程

GMMからデータを生成するプロセス：

1. **潜在変数のサンプリング**: $z \sim \text{Categorical}(\pi_1, ..., \pi_K)$
2. **データのサンプリング**: $\mathbf{x} \sim \mathcal{N}(\boldsymbol{\mu}_z, \Sigma_z)$

```
生成過程:

           π = [0.5, 0.3, 0.2]
                   ↓
         z ∼ Categorical(π)
                   ↓
              z = 2 を選択
                   ↓
         x ∼ N(μ₂, Σ₂)
                   ↓
              x = [1.2, 3.5]
```

### 📊 同時分布と周辺分布

**同時分布**:
$$p(\mathbf{x}, z=k) = \pi_k \mathcal{N}(\mathbf{x} | \boldsymbol{\mu}_k, \Sigma_k)$$

**周辺分布**（潜在変数を周辺化）:
$$p(\mathbf{x}) = \sum_{k=1}^{K} p(\mathbf{x}, z=k) = \sum_{k=1}^{K} \pi_k \mathcal{N}(\mathbf{x} | \boldsymbol{\mu}_k, \Sigma_k)$$

これが最初に定義したGMMの式です。

In [ ]:
# ============================================================
# GMMからのデータ生成
# 潜在変数を使った生成過程を実装します
# ============================================================

def sample_from_gmm(n_samples, weights, means, covs):
    """
    GMMからデータをサンプリング
    
    Parameters:
    -----------
    n_samples : int
        サンプル数
    weights : array-like
        混合係数（合計1）
    means : list of arrays
        各成分の平均ベクトル
    covs : list of arrays
        各成分の共分散行列
    
    Returns:
    --------
    samples : array, shape (n_samples, d)
        生成されたサンプル
    labels : array, shape (n_samples,)
        各サンプルの潜在変数（どの成分から生成されたか）
    """
    K = len(weights)
    d = len(means[0])
    
    samples = np.zeros((n_samples, d))
    labels = np.zeros(n_samples, dtype=int)
    
    for i in range(n_samples):
        # Step 1: 潜在変数をサンプリング
        z = np.random.choice(K, p=weights)
        labels[i] = z
        
        # Step 2: 選ばれた成分からデータをサンプリング
        samples[i] = np.random.multivariate_normal(means[z], covs[z])
    
    return samples, labels

# GMMのパラメータを設定（2次元、3成分）
np.random.seed(42)

weights = [0.4, 0.35, 0.25]
means = [
    np.array([0, 0]),
    np.array([4, 4]),
    np.array([-3, 4]),
]
covs = [
    np.array([[1, 0.5], [0.5, 1]]),
    np.array([[1.5, -0.5], [-0.5, 0.8]]),
    np.array([[0.8, 0], [0, 1.2]]),
]

# サンプリング
n_samples = 500
samples, labels = sample_from_gmm(n_samples, weights, means, covs)

# 可視化
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 左: 潜在変数あり（クラスタラベル表示）
colors = ['steelblue', 'coral', 'green']
for k in range(3):
    mask = labels == k
    axes[0].scatter(samples[mask, 0], samples[mask, 1], c=colors[k], 
                   alpha=0.6, s=30, label=f'成分 {k+1} (π={weights[k]})')
    axes[0].scatter(means[k][0], means[k][1], c=colors[k], s=200, 
                   marker='*', edgecolors='black', linewidths=2)

axes[0].set_xlabel('X₁', fontsize=12)
axes[0].set_ylabel('X₂', fontsize=12)
axes[0].set_title('潜在変数 z が見える場合\n（どの成分から生成されたか分かる）', fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].set_aspect('equal')

# 中央: 潜在変数なし（実際のデータ）
axes[1].scatter(samples[:, 0], samples[:, 1], c='gray', alpha=0.5, s=30)
axes[1].set_xlabel('X₁', fontsize=12)
axes[1].set_ylabel('X₂', fontsize=12)
axes[1].set_title('潜在変数 z が見えない場合\n（実際に観測されるデータ）', fontsize=13, fontweight='bold')
axes[1].set_aspect('equal')

# 右: 確率密度の等高線
x = np.linspace(-6, 8, 200)
y = np.linspace(-3, 8, 200)
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))

# GMM密度を計算
Z = np.zeros_like(X)
for w, mu, cov in zip(weights, means, covs):
    rv = stats.multivariate_normal(mu, cov)
    Z += w * rv.pdf(pos)

axes[2].contourf(X, Y, Z, levels=20, cmap='viridis', alpha=0.7)
axes[2].contour(X, Y, Z, levels=10, colors='white', alpha=0.5)
axes[2].scatter(samples[:, 0], samples[:, 1], c='white', alpha=0.3, s=10)

for k, mu in enumerate(means):
    axes[2].scatter(mu[0], mu[1], c='red', s=200, marker='*', 
                   edgecolors='white', linewidths=2)

axes[2].set_xlabel('X₁', fontsize=12)
axes[2].set_ylabel('X₂', fontsize=12)
axes[2].set_title('GMM確率密度関数\n（3つのピークがある）', fontsize=13, fontweight='bold')
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()

print("="*60)
print("GMMの生成過程")
print("="*60)
for k in range(3):
    count = np.sum(labels == k)
    print(f"成分 {k+1}: π={weights[k]:.2f} → 生成数 {count} ({count/n_samples*100:.1f}%)")

---

## 4. 事後確率（責任率）

### 🤔 事後確率とは？

データ点 $\mathbf{x}$ が観測されたとき、それがどの成分から生成されたかの確率を**事後確率**または**責任率（Responsibility）**と呼びます。

ベイズの定理より：

$$\gamma_{nk} = p(z=k | \mathbf{x}_n) = \frac{p(z=k) p(\mathbf{x}_n | z=k)}{p(\mathbf{x}_n)} = \frac{\pi_k \mathcal{N}(\mathbf{x}_n | \boldsymbol{\mu}_k, \Sigma_k)}{\sum_{j=1}^{K} \pi_j \mathcal{N}(\mathbf{x}_n | \boldsymbol{\mu}_j, \Sigma_j)}$$

- $\gamma_{nk}$: データ点 $n$ が成分 $k$ から生成された確率（責任率）
- 各データ点について、$\sum_{k=1}^{K} \gamma_{nk} = 1$

In [ ]:
# ============================================================
# 責任率の計算
# 各データ点がどの成分に属する確率を計算します
# ============================================================

def compute_responsibilities(X, weights, means, covs):
    """
    責任率（事後確率）を計算
    
    Parameters:
    -----------
    X : array, shape (n_samples, d)
        データ
    weights : array, shape (K,)
        混合係数
    means : list of arrays
        各成分の平均
    covs : list of arrays
        各成分の共分散行列
    
    Returns:
    --------
    responsibilities : array, shape (n_samples, K)
        責任率 γ_{nk}
    """
    n_samples = X.shape[0]
    K = len(weights)
    
    # 各成分での確率密度を計算
    weighted_pdfs = np.zeros((n_samples, K))
    for k in range(K):
        rv = stats.multivariate_normal(means[k], covs[k])
        weighted_pdfs[:, k] = weights[k] * rv.pdf(X)
    
    # 正規化（責任率）
    total_pdf = weighted_pdfs.sum(axis=1, keepdims=True)
    responsibilities = weighted_pdfs / total_pdf
    
    return responsibilities

# 責任率を計算
resp = compute_responsibilities(samples, weights, means, covs)

# 可視化
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 各成分への責任率で色付け
for k, (ax, color_name) in enumerate(zip(axes, ['Blues', 'Oranges', 'Greens'])):
    scatter = ax.scatter(samples[:, 0], samples[:, 1], c=resp[:, k], 
                        cmap=color_name, alpha=0.7, s=30)
    ax.scatter(means[k][0], means[k][1], c='red', s=200, marker='*', 
              edgecolors='black', linewidths=2)
    ax.set_xlabel('X₁', fontsize=12)
    ax.set_ylabel('X₂', fontsize=12)
    ax.set_title(f'成分 {k+1} への責任率 γ_{{n,{k+1}}}', fontsize=13, fontweight='bold')
    ax.set_aspect('equal')
    plt.colorbar(scatter, ax=ax, label='責任率')

plt.tight_layout()
plt.show()

# クラスタリング結果（責任率が最大の成分に割り当て）
predicted_labels = np.argmax(resp, axis=1)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: 真のラベル
for k in range(3):
    mask = labels == k
    axes[0].scatter(samples[mask, 0], samples[mask, 1], c=colors[k], 
                   alpha=0.6, s=30, label=f'成分 {k+1}')
axes[0].set_title('真のラベル', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].set_aspect('equal')

# 右: 推定ラベル
for k in range(3):
    mask = predicted_labels == k
    axes[1].scatter(samples[mask, 0], samples[mask, 1], c=colors[k], 
                   alpha=0.6, s=30, label=f'成分 {k+1}')
axes[1].set_title('責任率からの推定ラベル', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].set_aspect('equal')

plt.tight_layout()
plt.show()

# 精度を計算（ラベルの対応を考慮）
accuracy = np.mean(predicted_labels == labels)
print(f"\nクラスタリング精度: {accuracy*100:.1f}%")
print("（注: 今回はパラメータが既知なので高精度）")

---

## 5. GMMのパラメータ推定の難しさ

### 🤔 なぜ単純な最尤推定ができないのか？

GMMの対数尤度は：

$$\log p(X | \theta) = \sum_{n=1}^{N} \log \left( \sum_{k=1}^{K} \pi_k \mathcal{N}(\mathbf{x}_n | \boldsymbol{\mu}_k, \Sigma_k) \right)$$

**問題**: 対数の中に和があるため、微分して解析的に解くことができません！

単一の正規分布の場合：
$$\log p(X | \theta) = \sum_{n=1}^{N} \log \mathcal{N}(\mathbf{x}_n | \mu, \Sigma)$$

→ 対数と和が交換できるので、解析的に解ける

### 📊 直感的な理解

```
ニワトリと卵の問題:

- パラメータ (μ, Σ) を求めるには、各データがどの成分に属するか（z）を知る必要がある
- 各データがどの成分に属するか（z）を知るには、パラメータが必要

    ┌─────────────────┐
    │  パラメータ θ   │
    │  (π, μ, Σ)     │◄────────┐
    └────────┬────────┘         │
             │                   │
             ▼                   │
    ┌─────────────────┐         │
    │  責任率 γ_{nk}  │         │
    │  (事後確率)     │─────────┘
    └─────────────────┘
```

この循環を解決するのが、次章で学ぶ**EMアルゴリズム**です。

In [ ]:
# ============================================================
# パラメータ推定の難しさを体感
# ランダムな初期値から始めた場合の問題を可視化します
# ============================================================

np.random.seed(123)

# ランダムな初期パラメータ
random_weights = np.array([0.33, 0.33, 0.34])
random_means = [
    np.array([2, 2]),
    np.array([-2, 2]),
    np.array([0, -2]),
]
random_covs = [
    np.array([[1, 0], [0, 1]]),
    np.array([[1, 0], [0, 1]]),
    np.array([[1, 0], [0, 1]]),
]

# 責任率を計算
resp_random = compute_responsibilities(samples, random_weights, random_means, random_covs)
predicted_random = np.argmax(resp_random, axis=1)

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 左: 真のパラメータ
for k in range(3):
    mask = labels == k
    axes[0].scatter(samples[mask, 0], samples[mask, 1], c=colors[k], alpha=0.6, s=30)
    axes[0].scatter(means[k][0], means[k][1], c=colors[k], s=300, marker='*', 
                   edgecolors='black', linewidths=2)
axes[0].set_title('真のパラメータ\n(★ = 真の平均)', fontsize=14, fontweight='bold')
axes[0].set_aspect('equal')

# 中央: ランダムな初期パラメータ
axes[1].scatter(samples[:, 0], samples[:, 1], c='gray', alpha=0.5, s=30)
for k in range(3):
    axes[1].scatter(random_means[k][0], random_means[k][1], c=colors[k], s=300, 
                   marker='X', edgecolors='black', linewidths=2)
axes[1].set_title('ランダムな初期パラメータ\n(X = 初期平均)', fontsize=14, fontweight='bold')
axes[1].set_aspect('equal')

# 右: 初期パラメータでのクラスタリング結果
for k in range(3):
    mask = predicted_random == k
    axes[2].scatter(samples[mask, 0], samples[mask, 1], c=colors[k], alpha=0.6, s=30)
    axes[2].scatter(random_means[k][0], random_means[k][1], c=colors[k], s=300, 
                   marker='X', edgecolors='black', linewidths=2)
axes[2].set_title('初期パラメータでの結果\n→ 間違ったクラスタリング', fontsize=14, fontweight='bold')
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()

print("💡 ポイント: ランダムな初期パラメータでは正しくクラスタリングできません")
print("   → パラメータを反復的に更新する必要があります（EMアルゴリズム）")

---

## 6. scikit-learnでのGMM

### 📊 GaussianMixture クラス

scikit-learnの `GaussianMixture` クラスを使うと、EMアルゴリズムによるGMMのパラメータ推定が簡単にできます。

In [ ]:
# ============================================================
# scikit-learnでのGMM
# GaussianMixtureクラスを使ってパラメータを推定します
# ============================================================

from sklearn.mixture import GaussianMixture

# GMMモデルを作成
gmm_model = GaussianMixture(
    n_components=3,      # 成分数
    covariance_type='full',  # 共分散行列のタイプ
    n_init=10,           # 異なる初期値で10回試行
    random_state=42
)

# モデルをフィット
gmm_model.fit(samples)

# 推定されたパラメータ
print("="*60)
print("scikit-learn GaussianMixture の推定結果")
print("="*60)

print(f"\n【混合係数】")
print(f"  推定値: {gmm_model.weights_.round(3)}")
print(f"  真の値: {weights}")

print(f"\n【平均ベクトル】")
for k in range(3):
    print(f"  成分{k+1} 推定: {gmm_model.means_[k].round(3)}")
    # 対応する真の成分を見つける（最も近いもの）
    distances = [np.linalg.norm(gmm_model.means_[k] - m) for m in means]
    closest = np.argmin(distances)
    print(f"         真の値: {means[closest]}")

# クラスタリング結果
sklearn_labels = gmm_model.predict(samples)
sklearn_proba = gmm_model.predict_proba(samples)

# 可視化
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 左: sklearn の結果
for k in range(3):
    mask = sklearn_labels == k
    axes[0].scatter(samples[mask, 0], samples[mask, 1], c=colors[k], alpha=0.6, s=30)
    axes[0].scatter(gmm_model.means_[k, 0], gmm_model.means_[k, 1], 
                   c=colors[k], s=300, marker='*', edgecolors='black', linewidths=2)
axes[0].set_title('scikit-learn GMMの結果', fontsize=14, fontweight='bold')
axes[0].set_aspect('equal')

# 中央: 真のラベル
for k in range(3):
    mask = labels == k
    axes[1].scatter(samples[mask, 0], samples[mask, 1], c=colors[k], alpha=0.6, s=30)
    axes[1].scatter(means[k][0], means[k][1], c=colors[k], s=300, 
                   marker='*', edgecolors='black', linewidths=2)
axes[1].set_title('真のラベル', fontsize=14, fontweight='bold')
axes[1].set_aspect('equal')

# 右: 確率密度の等高線
x = np.linspace(-6, 8, 200)
y = np.linspace(-3, 8, 200)
X_grid, Y_grid = np.meshgrid(x, y)
XY = np.column_stack([X_grid.ravel(), Y_grid.ravel()])

# sklearnモデルのスコア（対数密度）
Z_sklearn = np.exp(gmm_model.score_samples(XY)).reshape(X_grid.shape)

axes[2].contourf(X_grid, Y_grid, Z_sklearn, levels=20, cmap='viridis', alpha=0.7)
axes[2].scatter(samples[:, 0], samples[:, 1], c='white', alpha=0.2, s=10)
for k in range(3):
    axes[2].scatter(gmm_model.means_[k, 0], gmm_model.means_[k, 1], 
                   c='red', s=200, marker='*', edgecolors='white', linewidths=2)
axes[2].set_title('推定されたGMM密度', fontsize=14, fontweight='bold')
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()

# 対数尤度
log_likelihood = gmm_model.score(samples) * len(samples)
print(f"\n対数尤度: {log_likelihood:.2f}")

In [ ]:
# ============================================================
# GMMによる新しいデータの生成
# 学習したモデルから新しいサンプルを生成します
# ============================================================

# 学習済みモデルからサンプル生成
generated_samples, generated_labels = gmm_model.sample(300)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左: 元のデータ
axes[0].scatter(samples[:, 0], samples[:, 1], c='steelblue', alpha=0.5, s=30, label='訓練データ')
axes[0].set_title('元のデータ', fontsize=14, fontweight='bold')
axes[0].set_aspect('equal')
axes[0].legend()

# 右: 生成データ
for k in range(3):
    mask = generated_labels == k
    axes[1].scatter(generated_samples[mask, 0], generated_samples[mask, 1], 
                   c=colors[k], alpha=0.6, s=30)
axes[1].set_title('GMMから生成されたデータ', fontsize=14, fontweight='bold')
axes[1].set_aspect('equal')

plt.tight_layout()
plt.show()

print("💡 学習したGMMを使って、元のデータと似た分布を持つ新しいデータを生成できます")
print("   これが『生成モデル』の力です！")

---

## 7. 成分数の選択

### 🤔 成分数Kをどう決めるか？

GMMでは、成分数Kを事前に指定する必要があります。一般的な方法：

1. **BIC（ベイズ情報量規準）**: $\text{BIC} = -2\log L + k\log n$
2. **AIC（赤池情報量規準）**: $\text{AIC} = -2\log L + 2k$

小さい値ほど良いモデル。パラメータ数が増えると罰則項が増加するため、過学習を防ぎます。

In [ ]:
# ============================================================
# 成分数の選択
# BICとAICを使って最適な成分数を選びます
# ============================================================

# 異なる成分数でモデルをフィット
n_components_range = range(1, 8)
bics = []
aics = []
log_likelihoods = []

for n_comp in n_components_range:
    gmm_temp = GaussianMixture(n_components=n_comp, random_state=42, n_init=5)
    gmm_temp.fit(samples)
    bics.append(gmm_temp.bic(samples))
    aics.append(gmm_temp.aic(samples))
    log_likelihoods.append(gmm_temp.score(samples) * len(samples))

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 左: BIC
axes[0].plot(n_components_range, bics, 'bo-', linewidth=2, markersize=8)
best_k_bic = np.argmin(bics) + 1
axes[0].axvline(x=best_k_bic, color='red', linestyle='--', label=f'最適 K={best_k_bic}')
axes[0].set_xlabel('成分数 K', fontsize=12)
axes[0].set_ylabel('BIC', fontsize=12)
axes[0].set_title('BIC（小さいほど良い）', fontsize=14, fontweight='bold')
axes[0].legend()

# 中央: AIC
axes[1].plot(n_components_range, aics, 'go-', linewidth=2, markersize=8)
best_k_aic = np.argmin(aics) + 1
axes[1].axvline(x=best_k_aic, color='red', linestyle='--', label=f'最適 K={best_k_aic}')
axes[1].set_xlabel('成分数 K', fontsize=12)
axes[1].set_ylabel('AIC', fontsize=12)
axes[1].set_title('AIC（小さいほど良い）', fontsize=14, fontweight='bold')
axes[1].legend()

# 右: 対数尤度
axes[2].plot(n_components_range, log_likelihoods, 'ro-', linewidth=2, markersize=8)
axes[2].axvline(x=3, color='blue', linestyle='--', label='真の成分数 K=3')
axes[2].set_xlabel('成分数 K', fontsize=12)
axes[2].set_ylabel('対数尤度', fontsize=12)
axes[2].set_title('対数尤度（大きいほど良いが...）', fontsize=14, fontweight='bold')
axes[2].legend()

plt.tight_layout()
plt.show()

print("="*60)
print("モデル選択の結果")
print("="*60)
print(f"BICによる最適成分数: K = {best_k_bic}")
print(f"AICによる最適成分数: K = {best_k_aic}")
print(f"真の成分数: K = 3")
print("\n💡 対数尤度だけでは成分数が増えるほど良くなるため、")
print("   BIC/AICの罰則項が過学習を防ぐのに重要です")

---

## まとめ

### 🎯 このノートブックで学んだこと

**多峰性分布**
- ✓ 単一の正規分布では表現できないデータがある
- ✓ 多峰性 = 複数のピークを持つ分布

**混合ガウスモデル（GMM）**
- ✓ 複数の正規分布の重み付き和: $p(\mathbf{x}) = \sum_{k=1}^{K} \pi_k \mathcal{N}(\mathbf{x} | \boldsymbol{\mu}_k, \Sigma_k)$
- ✓ 潜在変数 $z$ はどの成分から生成されたかを示す
- ✓ 責任率 $\gamma_{nk}$ = データ点が成分kに属する事後確率

**パラメータ推定**
- ✓ 対数尤度に和が含まれるため、解析的に解けない
- ✓ EMアルゴリズムで反復的に推定
- ✓ BIC/AICで成分数を選択

### 📊 GMMのチートシート

| 項目 | 説明 |
|------|------|
| モデル | $p(\mathbf{x}) = \sum_{k=1}^{K} \pi_k \mathcal{N}(\mathbf{x} \| \boldsymbol{\mu}_k, \Sigma_k)$ |
| 混合係数 | $\pi_k \geq 0$, $\sum_k \pi_k = 1$ |
| 責任率 | $\gamma_{nk} = \frac{\pi_k \mathcal{N}(\mathbf{x}_n \| \boldsymbol{\mu}_k, \Sigma_k)}{\sum_j \pi_j \mathcal{N}(\mathbf{x}_n \| \boldsymbol{\mu}_j, \Sigma_j)}$ |
| 成分数選択 | BIC, AIC（小さいほど良い）|

### 🔧 Pythonでの使い方

```python
from sklearn.mixture import GaussianMixture

# モデル作成とフィット
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(X)

# パラメータ
gmm.weights_      # 混合係数
gmm.means_        # 平均ベクトル
gmm.covariances_  # 共分散行列

# 予測
labels = gmm.predict(X)      # ハードクラスタリング
proba = gmm.predict_proba(X) # 責任率

# 生成
samples, labels = gmm.sample(n_samples)
```

---

### ⚠️ よくあるエラー

#### エラー #1: 成分数の選択ミス

```python
# ❌ 問題: 対数尤度だけで判断
best_k = max(range(1, 10), key=lambda k: GaussianMixture(k).fit(X).score(X))
# → 成分数を増やすほど対数尤度は上がる（過学習）

# ✅ 正しい: BIC/AICを使う
best_k = min(range(1, 10), key=lambda k: GaussianMixture(k).fit(X).bic(X))
```

#### エラー #2: 共分散行列の特異性

```python
# ❌ 問題: データが少ないときに共分散行列が特異になる
gmm = GaussianMixture(n_components=10, covariance_type='full').fit(X)
# → LinAlgError: singular matrix

# ✅ 解決策: 正則化を追加するか、共分散タイプを変更
gmm = GaussianMixture(n_components=10, covariance_type='diag', 
                      reg_covar=1e-6).fit(X)
```

---

## 🎓 自己評価クイズ

### Q1: GMMの混合係数 $\pi_k$ は何を表しますか？

<details>
<summary>💡 答えを見る</summary>

**答え**: 各成分から生成される確率（事前確率）

$\pi_k = P(z = k)$ は、新しいデータ点が成分 $k$ から生成される確率です。
すべての $\pi_k$ の和は1になります。

</details>

---

### Q2: 責任率 $\gamma_{nk}$ と混合係数 $\pi_k$ の違いは？

<details>
<summary>💡 答えを見る</summary>

**答え**:

- **$\pi_k$（事前確率）**: データを見る前の、成分 $k$ から生成される確率
- **$\gamma_{nk}$（事後確率）**: データ $\mathbf{x}_n$ を観測した後の、成分 $k$ から生成された確率

責任率はベイズの定理で計算され、観測データの情報を反映しています。

</details>

---

### Q3: なぜGMMの最尤推定は解析的に解けないのですか？

<details>
<summary>💡 答えを見る</summary>

**答え**: 対数尤度の中に和があるため

$$\log p(X) = \sum_n \log \left( \sum_k \pi_k \mathcal{N}(\mathbf{x}_n | \mu_k, \Sigma_k) \right)$$

対数と和の順序を交換できないため、微分して0とおいても解析解が得られません。
これがEMアルゴリズムが必要な理由です。

</details>

---

### Q4: GMMとK-meansクラスタリングの違いは？

<details>
<summary>💡 答えを見る</summary>

**答え**:

| 項目 | K-means | GMM |
|------|---------|-----|
| 割り当て | ハード（0 or 1）| ソフト（確率）|
| クラスタ形状 | 球状のみ | 楕円形も可能 |
| 密度モデル | なし | あり（生成可能）|
| 不確実性 | 表現不可 | 責任率で表現 |

GMMはK-meansの確率的な拡張と見なせます。

</details>

---

### 📚 練習問題

1. **異なる混合係数**: 混合係数を [0.7, 0.2, 0.1] に変更してデータを生成し、GMMで推定できるか確認してください。

2. **次元の追加**: 3次元のGMM（例: 身長、体重、胸囲）を実装し、可視化してください。

3. **収束の観察**: EMアルゴリズムの各イテレーションでの対数尤度をプロットし、収束を観察してください（sklearn の `warm_start=True` を使用）。

---

### ✅ 学習チェックリスト

- [ ] 多峰性分布と単一正規分布の違いを説明できる
- [ ] GMMの確率密度関数を書ける
- [ ] 潜在変数と生成過程を理解している
- [ ] 責任率の計算ができる
- [ ] パラメータ推定が難しい理由を説明できる

---

**次のステップ**: Notebook 34で、**EMアルゴリズム**を学びます！

GMMのパラメータを推定するための反復アルゴリズムであるEMアルゴリズムを、理論と実装の両面から学習します。これはVAEの理解にも不可欠な概念です。